In [ ]:
HUME_API_KEY = "1Fuo6eVLpIj6ndhmC5VXllArH67eOcaSA0XLX3sHdU2SdEy5"

In [ ]:
from pathlib import Path

TEXT = "I'm so excited for the camping trip next weekend!"

filepath = "text.txt"
with open(filepath, "w") as fp:
    fp.write(TEXT)

In [ ]:
client = HumeBatchClient(HUME_API_KEY)
config = LanguageConfig(granularity="sentence", identify_speakers=True)
job = client.submit_job(None, [config], files=[filepath])

print("Running...", job)

job.await_complete()
print("Job completed with status: ", job.get_status())